In [1]:
import os
print (os.getcwd())

/Users/apil.tamang/Git/VGG-Experiments


In [2]:
from os.path import join
# file paths to training and validation samples

gFilesBase = "/Users/apil.tamang/kaggle_cervical_cancer_code/segmented_src"
filepaths = []
filepaths.append( join(gFilesBase, "train/Type_1/") )
filepaths.append( join(gFilesBase, "train/Type_2/") )
filepaths.append( join(gFilesBase, "train/Type_3/") )

filepaths.append( join(gFilesBase, "valid/Type_1/") )
filepaths.append( join(gFilesBase, "valid/Type_2/") )
filepaths.append( join(gFilesBase, "valid/Type_3/") )

# Get list of all the training images
import glob
allFiles = []

for i, filepath in enumerate(filepaths):
    files = glob.glob(filepath + '*.jpg')
    allFiles = allFiles + files

print(len(allFiles))

7627


In [3]:
train_valid_fraction = 0.8

In [4]:
### Create training and validation list by splitting data based on specified *train_valid_fraction*

import random
split_point = int(train_valid_fraction*len(allFiles))

random.shuffle(allFiles)

train_list = allFiles[:split_point]
valid_list = allFiles[split_point:]
print('Train patients: {}'.format(len(train_list)))
print('Valid patients: {}'.format(len(valid_list)))

Train patients: 6101
Valid patients: 1526


In [5]:
import cv2
import numpy as np

def batch_generator_train2(files):
    random.shuffle(files)
    image_list = []
    mask_list = []
    for f in files:
        image = cv2.imread(f)

        #commenting this out, because the shape of the
        #image is already set to desired (224x224)
        i#mage = cv2.resize(image, conf['image_shape'])

        if "Type_1" in f:
            mask = [1, 0, 0]
        elif "Type_2" in f:
            mask = [0, 1, 0]
        elif "Type_3" in f:
            mask = [0, 0, 1]
        else:
            raise RuntimeError("Bad file name, couldn't determine cancer type")

        image_list.append(image)
        mask_list.append(mask)

    image_list = np.array(image_list)
    mask_list = np.array(mask_list)
    return (image_list, mask_list)

def GetDataFromFileLists(trainList, validList):
    (X_train, Y_train) = batch_generator_train2(trainList)
    (X_valid, Y_valid) = batch_generator_train2(validList)
    return (X_train, Y_train), (X_valid, Y_valid)
            
           
(X_train, Y_train), (X_valid, Y_valid) = GetDataFromFileLists(train_list, valid_list)

In [7]:
def preprocess_input(x):
    x[:, :, :, 0] -= 103.939
    x[:, :, :, 1] -= 116.779
    x[:, :, :, 2] -= 123.68
    # 'RGB'->'BGR'
    x = x[:, :, :, ::-1]
    return x


In [8]:
X_train = preprocess_input(X_train.astype(np.float))
X_valid = preprocess_input(X_valid.astype(np.float))

In [9]:
import numpy as np
np.save("X_train.224x224.npy",X_train)
np.save("X_valid.224x224.npy",X_valid)
np.save("Y_train.224x224.npy",Y_train)
np.save("Y_valid.224x224.npy",Y_valid)